快速入门langchain的教程

## LLM链

In [1]:
! export LANGCHAIN_TRACING_V2="true"
! export LANGCHAIN_API_KEY="..."

### 语言模型

In [2]:
from langchain_community.llms import Ollama
llm = Ollama(model="qwen:7b",
             base_url="http://192.168.1.10:11434",
             num_gpu=1)
             
llm.invoke("LangSmith是什么")

'"Langsmith"并不是一个普遍熟知的名字或事物，因此无法提供具体的信息。如果这是一个人名，可能需要更多的上下文来确定其身份或含义。'

### 聊天模型的提示模板

用于为聊天模型创建灵活的模板提示。

[langchain_core.prompts.chat.ChatPromptTemplate](langchain_core.prompts.chat.ChatPromptTemplate.ipynb)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位世界一流的技术文档作家。"),
    ("user", "{input}")
])
chain = prompt | llm 
chain.invoke({"input": "Langsmith如何帮助测试？"})

'Langsmith，这个名字通常与语言处理或者人工智能领域的一个工具或模型关联，而不是一个直接帮助测试的实体。\n\n如果是指一个名为 Langsmith 的测试辅助工具，那它可能通过以下方式帮助测试：\n\n1. **代码覆盖率分析**：Langsmith能自动检测代码中的语句和分支是否被正确地执行，从而提供关于测试覆盖率的信息。\n\n2. **性能基准测试**：针对特定功能或系统，Langsmith可以模拟真实用户行为进行大规模的性能基准测试，以此评估产品在不同负载下的表现。\n\n3. **缺陷跟踪与管理**：Langsmith能帮助识别和记录测试过程中发现的问题，方便团队进行问题追踪和解决。\n\n总之，如果Langsmith是一个工具或者模型，它可以通过自动化测试、代码分析、性能评估等多种方式来辅助测试工作。'

### 输出解析器

用于解析LLM结果到最可能字符串

[langchain_core.output_parsers.string.StrOutputParser](langchain_core.output_parsers.string.StrOutputParser.ipynb)

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "Langsmith如何帮助测试？"})

'Langsmith可能并不是一个明确的工具或服务，但如果你是在询问关于语言翻译和辅助测试的一般方法，那么我可以给出如下解释：\n\n1. **语言翻译**：在测试环境中，如果需要对非母语的测试人员或者自动化脚本进行支持，可以利用Langsmith这样的翻译工具。它能够实时将源语言文本转化为目标语言，从而帮助理解、编写或校对多语言测试内容。\n\n2. **辅助测试**：有时候，虽然文字翻译没问题，但为了确保测试过程的清晰和可重复性，Langsmith或者类似的工具可能还不能直接应用。这时，可能需要结合专业的测试管理软件（如JIRA、TestRail等）来协调和记录各种语言的测试任务。\n\n总结来说，Langsmith在帮助多语言测试中可以作为一个辅助工具，但要确保测试过程的完整性和可重复性，可能还需要与其他专业测试工具和服务相结合。'

### 小结

至此，讨论了基本的LLM链，涉及语言模型、提示、输出解析器，更深入的部分，参考

[模型I/O](模型IO.ipynb)

## 检索链



In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

# 创建提示模板
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# 创建填充文档链，调用它的invoke方法时，需要传入字典，键为提示中的变量，值为提示中变量的值
document_chain = create_stuff_documents_chain(llm, prompt)

# 直接将Document实例传入上下文
document_chain.invoke({
    "input": "Langsmith如何帮助测试？",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith通过可视化功能帮助测试。这意味着它可能提供了一种图形化的方式来展示测试的结果，使得测试结果更加直观易懂。'

## 对话链

In [6]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


# 加载语言模型 
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html
llm = Ollama(model="qwen:7b",
             base_url="http://192.168.1.10:11434",
             num_gpu=1)

# 构建知识库
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
# https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.ollama.OllamaEmbeddings.html
embeddings = OllamaEmbeddings(
    model="shaw/dmeta-embedding-zh:latest",
    base_url="http://192.168.1.10:11434",
    num_gpu=1
)
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# 首先创建一个提示，接收聊天记录、用户输入
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
# 创建检索器
retriever2 = create_history_aware_retriever(llm, retriever, prompt)
# 再创建一个提示，接收检索返回、聊天记录、用户输入
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
# 创建文档填充链
document_chain = create_stuff_documents_chain(llm, prompt)
# 创建检索链
retrieval_chain = create_retrieval_chain(retriever2, document_chain)


In [7]:
from pprint import pprint 


chat_history = [HumanMessage(content="LangSmith 可以帮助测试我的 LLM 应用吗？"), 
                AIMessage(content="是的!")]
pprint(retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "告诉我怎样做"
}))

{'answer': '要利用 LangSmith 进行 LLM 应用程序的测试，你可以按照以下步骤操作：\n'
           '\n'
           '1. **设置日志系统**：\n'
           '   首先，在你的 LLM 应用中集成 LangSmith。确保你有正确的 API 密钥或其他认证方式来与 LangSmith '
           '服务交互。\n'
           '\n'
           '2. **配置测试环境**：\n'
           '   根据你的应用需求，确定需要进行哪些方面的测试。这可能包括对特定模型的性能、处理不同输入数据的能力等方面。\n'
           '\n'
           '3. **编写和运行测试用例**：\n'
           '   使用 LangSmith 提供的 API 或 SDK '
           '编写自动化测试脚本。这些脚本可以在不同的场景下运行，比如模拟用户对话，或者针对模型的特定输入进行验证。\n'
           '\n'
           '4. **分析和调试结果**：\n'
           '   在测试过程中，可能会发现一些预期之外的问题或错误。这时，你可以使用 LangSmith 的追踪和调试功能来定位问题并解决。\n'
           '\n'
           '通过以上步骤，你就可以利用 LangSmith 来进行全面而细致的 LLM 应用程序测试了。',
 'chat_history': [HumanMessage(content='LangSmith 可以帮助测试我的 LLM 应用吗？'),
                  AIMessage(content='是的!')],
 'context': [Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'descrip

In [8]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever2,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [9]:
import getpass
import os

# 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
os.environ["TAVILY_API_KEY"] = getpass.getpass()

from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

retriever.invoke("今天天气怎么样？")

[Document(page_content='查询全国各地今天天气情况，包括温度、降水、风力、湿度、气压等指标。北京今天25℃/13℃，晴，微风，相对湿度31%，气压1004hpa。江南东部和华南等地有强降水和强对流，内蒙古宁夏华北西部等地有大风降温。', metadata={'title': '中国气象局-天气预报', 'source': 'https://weather.cma.cn/', 'score': 0.95843, 'images': None}),
 Document(page_content='2345天气预报提供全国3259个城区及全球40天天气预报查询服务，可以查询今天的天气情况，如多云，温度10~23°，空气优，紫外线很弱。还可以查看未来15天和40天的天气变化，以及生活指数、交通指数、旅行出游指数等专业天气预报资讯。', metadata={'title': '天气,天气预报查询,24小时,今天,明天,未来一周7天,10天,15天,40天查询_2345天气预报', 'source': 'https://tianqi.2345.com/', 'score': 0.93195, 'images': None}),
 Document(page_content='天气网提供全国各地的天气预报、气象公告、生活养生、旅游景点等信息。查询今天天气怎么样，可以查看全国24小时降水量预报、全国入春进程图、全国主要公路气象预报等。', metadata={'title': '天气网', 'source': 'http://www.weather.com.cn/', 'score': 0.9051, 'images': None})]